# Gradient Analysis for BloomZ-560M with LoRA

This notebook provides an interactive environment for analyzing gradients and activations during the fine-tuning of the BloomZ-560M model using LoRA. We will visualize the gradients, compute their similarity, and analyze the Pearson correlation coefficient between different training iterations.

In [1]:
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from src.utils.gradient_similarity import calculate_gradient_similarity
from src.utils.metrics import compute_pearson_correlation
from src.utils.data_loader import get_data_loaders
from src.models.lora_model import LoRAModel

# Set up the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# Load data
train_loader, val_loader = get_data_loaders(batch_size=32)

# Initialize the LoRA model
model = LoRAModel().to(device)

In [3]:
# Function to analyze gradients
def analyze_gradients(model, data_loader, num_iterations):
    gradient_history = []
    for iteration in range(num_iterations):
        # Simulate training step
        for inputs, targets in data_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            model.zero_grad()
            outputs = model(inputs)
            loss = torch.nn.functional.cross_entropy(outputs, targets)
            loss.backward()
            gradients = [param.grad.data.clone() for param in model.parameters() if param.grad is not None]
            gradient_history.append(gradients)
            break  # For demonstration, break after one batch
    return gradient_history

In [4]:
# Analyze gradients over a number of iterations
num_iterations = 10
gradient_history = analyze_gradients(model, train_loader, num_iterations)

In [5]:
# Calculate gradient similarity
similarity_results = []
for i in range(1, len(gradient_history)):
    similarity = calculate_gradient_similarity(gradient_history[i-1], gradient_history[i])
    similarity_results.append(similarity)
    print(f'Gradient similarity between iteration {i-1} and {i}: {similarity}')

In [6]:
# Compute Pearson correlation coefficient
pearson_results = []
for i in range(1, len(gradient_history)):
    correlation = compute_pearson_correlation(gradient_history[i-1], gradient_history[i])
    pearson_results.append(correlation)
    print(f'Pearson correlation between iteration {i-1} and {i}: {correlation}')

In [7]:
# Visualize the results
plt.figure(figsize=(12, 6))
sns.lineplot(data=similarity_results, label='Gradient Similarity')
plt.title('Gradient Similarity Over Iterations')
plt.xlabel('Iteration')
plt.ylabel('Similarity')
plt.legend()
plt.show()

In [8]:
# Visualize Pearson correlation results
plt.figure(figsize=(12, 6))
sns.lineplot(data=pearson_results, label='Pearson Correlation')
plt.title('Pearson Correlation Over Iterations')
plt.xlabel('Iteration')
plt.ylabel('Correlation Coefficient')
plt.legend()
plt.show()